# Expand the dataset by classifying using Mobilenetv2

## 2024-08-01


In [47]:
import cv2
import os
import numpy as np
import pandas as pd

In [48]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc("font", size=14)
plt.rc("axes", labelsize=14, titlesize=14)
plt.rc("legend", fontsize=14)
plt.rc("xtick", labelsize=10)
plt.rc("ytick", labelsize=10)

In [49]:
import tensorflow as tf
import keras

tf.random.set_seed(42)
keras.backend.clear_session()

In [ ]:
# my_list = ["discocyte","echinocyte", "granular", "holly_leaf", "sickle"]
# root_path = "/home/t.afanasyeva/deep_learning_anaemias/resources/out/cytpix/24-711122_sorted"
# for items in my_list:
#     path = os.path.join(root_path, items)
#     os.mkdir(path)

In [56]:
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, ReLU
from keras.regularizers import L2


mobile_net = keras.applications.MobileNetV2(
    input_shape=(64, 64, 3),
    alpha=1.0,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=5,
    classifier_activation="softmax",
)

model = Sequential()

model.add(mobile_net)
mobile_net.trainable = True

model.add(Dense(1024, kernel_regularizer=L2(0.01)))
model.add(BatchNormalization())
model.add(ReLU())

model.add(Dense(1024, kernel_regularizer=L2(0.01)))
model.add(BatchNormalization())
model.add(ReLU())

model.add(Dense(512, kernel_regularizer=L2(0.01)))
model.add(BatchNormalization())
model.add(ReLU())

model.add(GlobalAveragePooling2D())
model.add(Dense(5, activation="softmax"))  # Final layer with softmax activation


model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"],
)
model.build((None, 64, 64, 3))
model.summary()

/tmp/ipykernel_1802232/3544968198.py:7: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobile_net = keras.applications.MobileNetV2(


In [58]:
model.load_weights(
    "/home/t.afanasyeva/deep_learning_anaemias/model_zoo/mobile_net_v02.weights.h5",
    skip_mismatch=False,
)

/home/t.afanasyeva/miniforge3/envs/IMG/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 342 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Predicting new cytopix images

In [85]:
path_in = "/home/t.afanasyeva/deep_learning_anaemias/resources/out/cytpix/24-711122"
fnames = [path_in + "/" + image for image in os.listdir(path_in)]

In [88]:
rescale = tf.keras.Sequential([Rescaling(1.0 / 255)])

X_test = []
for name in fnames:
    img = cv2.imread(name)
    img = rescale(img)
    X_test.append(img)

In [112]:
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE


def prepare(list_of_images):
    df = tf.data.Dataset.from_tensor_slices(tf.convert_to_tensor(list_of_images))
    df = df.shuffle(len(df))
    df = df.batch(
        BATCH_SIZE,
        drop_remainder=True,
        num_parallel_calls=AUTOTUNE,
    )
    return df.prefetch(buffer_size=AUTOTUNE)

In [113]:
df = prepare(X_test)

In [114]:
y_pred = model.predict(df)

464/464 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step


In [115]:
y_pred = y_pred.argmax(axis=1)

In [116]:
len(y_pred)

14848

In [121]:
name_dict = {
    0: "discocyte",
    1: "holly_leaf",
    2: "granular",
    3: "sickle",
    4: "echinocyte",
}

In [123]:
y_pred = [name_dict[k] for k in y_pred]

In [124]:
path_out = (
    "/home/t.afanasyeva/deep_learning_anaemias/resources/out/cytpix/24-711122_sorted"
)

In [126]:
for name, prediction in zip(fnames, y_pred):
    file_name = name.split("/")[-1]
    img = cv2.imread(os.path.join(path_in, name))
    cv2.imwrite(os.path.join(path_out, prediction, file_name), img)

******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******
******